In [1]:
import sys
from pathlib import Path
from torch.utils.data import DataLoader, Subset


project_root = Path.cwd().parent
sys.path.append(str(project_root))

from src.data.dataset import ISICSkinDataset
from src.data.transforms import train_transforms, val_transforms


train_dataset = ISICSkinDataset(
    csv_file=project_root / "data/processed/train/train_binary.csv",
    image_dir=project_root / "data/raw/train/images_train",
    transform=train_transforms
)

val_dataset = ISICSkinDataset(
    csv_file=project_root / "data/processed/val/val_binary.csv",
    image_dir=project_root / "data/raw/val/images_val",
    transform=val_transforms
)

train_subset = Subset(train_dataset, range(1000))

train_loader = DataLoader(
    train_subset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

val_loader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=0
)



print(len(train_dataset))  # 9885
print(len(val_dataset))    # 193


9885
193


ResNet-50 Setup (Model)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


ResNet-50 Defination

In [4]:
from torchvision.models import ResNet50_Weights

model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Replace final layer (binary classification)
model.fc = nn.Linear(model.fc.in_features, 2)

model = model.to(device)



In [5]:
# Freeze everything
for param in model.parameters():
    param.requires_grad = False

# Unfreeze classifier
for param in model.fc.parameters():
    param.requires_grad = True

# Unfreeze last ResNet block
for param in model.layer4.parameters():
    param.requires_grad = True



In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-5)


In [7]:
epochs = 5

for epoch in range(epochs):

    # ---------- TRAIN ----------
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

    train_loss /= train_total
    train_acc = train_correct / train_total

    # ---------- VALIDATION ----------
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_loss /= val_total
    val_acc = val_correct / val_total

    print(
        f"Epoch {epoch+1}/{epochs} | "
        f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
        f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
    )


Epoch 1/5 | Train Loss: 0.5413, Train Acc: 0.8200 | Val Loss: 0.4747, Val Acc: 0.7720
Epoch 2/5 | Train Loss: 0.4224, Train Acc: 0.8300 | Val Loss: 0.4144, Val Acc: 0.7720
Epoch 3/5 | Train Loss: 0.3832, Train Acc: 0.8300 | Val Loss: 0.3841, Val Acc: 0.7720
Epoch 4/5 | Train Loss: 0.3538, Train Acc: 0.8310 | Val Loss: 0.3722, Val Acc: 0.7772
Epoch 5/5 | Train Loss: 0.3312, Train Acc: 0.8320 | Val Loss: 0.3678, Val Acc: 0.7824


In [8]:
# Freeze all model parameters
for param in model.parameters():
    param.requires_grad = False

print("✅ Model frozen: all parameters require_grad = False")


✅ Model frozen: all parameters require_grad = False


In [9]:
# Sanity check: count trainable parameters
trainable_params = sum(p.requires_grad for p in model.parameters())
total_params = sum(1 for _ in model.parameters())

print(f"Trainable params: {trainable_params} / {total_params}")


Trainable params: 0 / 161


In [ ]:
from pathlib import Path

# Create directory if not exists
weights_dir = Path("saved_models")
weights_dir.mkdir(exist_ok=True)

# Save model state
model_path = weights_dir / "resnet50_phase1_frozen.pth"
torch.save(model.state_dict(), model_path)

print(f"✅ Model weights saved at: {model_path}")

#model lading code
#model.load_state_dict(torch.load("saved_models/resnet50_phase1_frozen.pth"))
#model.eval()


✅ Model weights saved at: saved_models\resnet50_phase1_frozen.pth
